In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [47]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
dataframe =[train_df,test_df]
for i in dataframe:
    i['Sex'] = i['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
    i['Embarked'] = i['Embarked'].fillna('S')
    i['Cabin']=i['Cabin'].fillna('0')
    i.drop(['PassengerId','Ticket'],axis = 1, inplace=True)

In [48]:
train_df.head(2)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,7.2500,0,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,71.2833,C85,C


In [69]:
train_df.Name.str.split().str[2].len(2) .unique()

AttributeError: 'Series' object has no attribute 'len'

In [57]:
train_df.loc[train_df.Name.str.split()]

TypeError: unhashable type: 'list'